In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import StandardScaler,power_transform
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from imblearn.over_sampling import SMOTE
import pickle


In [ ]:
test=pd.read_csv('test_loan_data.csv')
test.head()

In [ ]:
Best_featurs=['addr_state', 'annual_inc', 'earliest_cr_line', 'emp_title', 'int_rate',
       'loan_amnt', 'tot_cur_bal', 'open_acc', 'revol_bal', 'revol_util',
       'sub_grade', 'total_acc']

In [ ]:
df=test[Best_featurs]
df.head()

In [ ]:
df.shape

In [ ]:
df.dropna(inplace=True)

In [ ]:
numerical_columns = df.select_dtypes(include=['float64','int64']).columns
categorical_columns = df.select_dtypes(include=['object']).columns

In [ ]:
numerical_columns

In [ ]:
categorical_columns

In [ ]:
# Handle outliers in numerical columns using IQR method
def handle_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df[column] = np.where((df[column] < lower_bound) | (df[column] > upper_bound),
                          df[column].median(), df[column])
    return df




In [ ]:
for colum in numerical_columns:
    handle_outliers(df,colum)

In [ ]:
import matplotlib.pyplot as plt

for i in numerical_columns:
    plt.figure(figsize=(10, 6))
    sns.boxplot(data=df, x=i)
    plt.title(f'Box plot of {i}')
    plt.show()

# lanel encoding

In [ ]:
df[categorical_columns]

In [ ]:
for col in categorical_columns:
    lebe_encoder=LabelEncoder()
    df[col]=lebe_encoder.fit_transform(df[col])

In [ ]:
df.head()

In [ ]:
sacale=StandardScaler()
df[numerical_columns]=sacale.fit_transform(df[numerical_columns])


In [ ]:
df.head()

In [ ]:
col = df.columns
col

In [ ]:
# Plotting the distribution of the variables (skewness) of all the columns
k=0
plt.figure(figsize=(17,28))
for col in col :
    k=k+1
    plt.subplot(6, 5,k)
    sns.distplot(df[col])
    plt.title(col+' '+str(df[col].skew()))

In [ ]:
from sklearn.preprocessing import PowerTransformer

# Identify numerical columns
# numerical_columns = df.select_dtypes(include=['number']).columns.tolist()

# Apply Yeo-Johnson transformation
pt = PowerTransformer(method='yeo-johnson')
df_yeojohnson_transformed = df.copy()

for col in numerical_columns:
    df_yeojohnson_transformed[[col]] = pt.fit_transform(df_yeojohnson_transformed[[col]])

df_yeojohnson_transformed.head()


In [ ]:
model=pickle.load(open('model_logi.pkl','rb'))


In [ ]:
pred=model.predict(df_yeojohnson_transformed)
pred.T

In [ ]:
predicted_categories = lebe_encoder.inverse_transform(pred)

result = pd.DataFrame(predicted_categories, columns=['loan_status'])
result.head()